ΚΑΡΛΗΣ ΝΙΚΟΛΑΟΣ 1115201800068


ΧΑΤΖΗΠΑΥΛΟΥ ΜΑΡΙΑ 1115201400223

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Διάβασμα του αρχείου με μετατροπή σε dataframe




In [ ]:
# read files
df = pd.read_csv('/content/gdrive/MyDrive/nasdaq2007_17.csv', header=None,sep = '\t')

df = df.reset_index(drop=True)

print("Number of rows and columns:", df.shape)


Number of rows and columns: (359, 3651)


το df.shape[0] είναι το πλήθος των γραμμών και το df.shape[1] είναι το πλήθος των στηλών. 

Αφού πρέπει να χωρίσουμε κάθε time series σε 80% training set και 20% test set βρίσκουμε σε ποιο column του dataframe αντιστοιχεί το 80% του time series που είναι το df.shape[1] * 0.8

Αποθηκεύουμε το αποτέλεσμα στην μεταβλητή percentage

In [ ]:
rows = df.shape[0]
columns = df.shape[1] -1

percentage = int(columns * 0.80)


In [ ]:
startdate="01/01/2015"
window_length = 10
encoding_dim = 3
epochs = 100
test_samples = 2000

In [ ]:
def mkdate(ts):
    return datetime.datetime.fromtimestamp(
        int(ts)
    ).strftime('%Y-%m-%d')

def plot_examples(stock_input, stock_decoded):
    n = 10  
    plt.figure(figsize=(20, 4))
    for i, idx in enumerate(list(np.arange(0, test_samples, 200))):
        # display original
        ax = plt.subplot(2, n, i + 1)
        if i == 0:
            ax.set_ylabel("Input", fontweight=600)
        else:
            ax.get_yaxis().set_visible(False)
        plt.plot(stock_input[idx])
        ax.get_xaxis().set_visible(False)
        

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        if i == 0:
            ax.set_ylabel("Output", fontweight=600)
        else:
            ax.get_yaxis().set_visible(False)
        plt.plot(stock_decoded[idx])
        ax.get_xaxis().set_visible(False)
        
        
def plot_history(history):
    plt.figure(figsize=(15, 5))
    ax = plt.subplot(1, 2, 1)
    plt.plot(history.history["loss"])
    plt.title("Train loss")
    ax = plt.subplot(1, 2, 2)
    plt.plot(history.history["val_loss"])
    plt.title("Test loss")

Σκεφτήκαμε να εφαρμόσουμε τον κώδικα που μας δόθηκε αλλά αντί για ένα dataframe με μία στήλη με πολλές γραμμές έχουμε πολλές στήλες με πολλές γραμμές.


Ξεκινήσαμε να κάνουμε iterate κάθε γραμμή του dataframe με το iterrows(). Παρατηρήσαμε ότι με την χρήση του row εμφανίζονταν κάθετα τα δεδομένα της γραμμής, σαν στήλη, επομένως σε κάθε επανάληψη δημιουργούμε ένα temp dataframe που έχει τα time series κάθε γραμμής. 

Έτσι το dataframe temp είναι σαν αυτό που έχει στο παράδειγμα και μπορούμε δημιουργήσουμε σε κάθε επανάληψη τα αντίστοιχα X_train και X_test, και τα αποθηκεύουμε σε αντίστοιχες λίστες trainList και testList.

Το scale γίνεται ξεχωριστά σε κάθε γραμμή του dataframe

In [ ]:
scaler = MinMaxScaler()

trainList = []
testList = []

for index, row in df.iterrows():
  temp = pd.DataFrame(row[1:].astype(float))
  temp.columns = ['price']

  temp['pct_change'] = temp.price.pct_change()
  temp['log_ret'] = np.log(temp.price) - np.log(temp.price.shift(1))
  x_train_nonscaled = np.array([temp['log_ret'].values[i-window_length:i].reshape(-1, 1) for i in tqdm(range(window_length+1,len(temp['log_ret'])))])
  x_train = np.array([scaler.fit_transform(temp['log_ret'].values[i-window_length:i].reshape(-1, 1)) for i in tqdm(range(window_length+1,len(temp['log_ret'])))])
  
  x_test = x_train[percentage:]
  x_train = x_train[:percentage]

  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  trainList.append(x_train)
  testList.append(x_test)

100%|██████████| 3639/3639 [00:00<00:00, 6252.73it/s]


In [ ]:
trainShape = trainList[0].shape
print(trainShape)
print("The length of train list is: ", len(trainList))

(2920, 10, 1)
The length of train list is:  359


In [ ]:
testShape = testList[0].shape
print(testShape)

print("The length of test list is: ", len(testList))

(719, 10, 1)
The length of test list is:  359


Το μοντέλο που μας δόθηκε.

Αλλάξαμε το activation function σε tanh, διότι έχουμε κανονικοποιήσει τα δεδομένα με το minmaxscaler. Επομένως το tanh είναι καλύτερο από το sigmoid


In [ ]:
input_window = Input(shape=(window_length,1))
x = Conv1D(16, 3, activation="tanh", padding="same")(input_window) # 10 dims
#x = BatchNormalization()(x)
x = MaxPooling1D(2, padding="same")(x) # 5 dims
x = Conv1D(1, 3, activation="tanh", padding="same")(x) # 5 dims
#x = BatchNormalization()(x)
encoded = MaxPooling1D(2, padding="same")(x) # 3 dims

encoder = Model(input_window, encoded)

# 3 dimensions in the encoded layer

x = Conv1D(1, 3, activation="tanh", padding="same")(encoded) # 3 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x) # 6 dims
x = Conv1D(16, 2, activation='tanh')(x) # 5 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x) # 10 dims
decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x) # 10 dims
autoencoder = Model(input_window, decoded)
autoencoder.summary()



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 10, 16)            64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 5, 16)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 5, 1)              49        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 3, 1)             0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 3, 1)              4   

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


for x_train, x_test in zip(trainList, testList): #10
  history = autoencoder.fit(x_train, x_train, epochs=10, batch_size=1024,
                shuffle=True, validation_data=(x_test, x_test))


Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 29ms/step - loss: 0.6305 - val_loss: 0.6319
Epoch 2/10
3/3 [==============================] - 0s 18ms/step - loss: 0.6304 - val_loss: 0.6318
Epoch 3/10
3/3 [==============================] - 0s 22ms/step - loss: 0.6305 - val_loss: 0.6318
Epoch 4/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6305 - val_loss: 0.6318
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6304 - val_loss: 0.6317
Epoch 6/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6305 - val_loss: 0.6317
Epoch 7/10
3/3 [==============================] - 0s 19ms/step - loss: 0.6304 - val_loss: 0.6317
Epoch 8/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6305 - val_loss: 0.6317
Epoch 9/10
3/3 [==============================] - 0s 16ms/step - loss: 0.6305 - val_loss: 0.6317
Epoch 10/10
3/3 [==============================] - 0s 18ms/step - loss: 0.6305 - val_lo

In [ ]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
autoencoder.save('/content/gdrive/MyDrive/model3')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/model3/assets


In [ ]:
encoder.save('/content/gdrive/MyDrive/encoder')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/encoder/assets


Για κάθε X_test στο testList αποθηκεύουμε τα prediction του encoder σε μια encodedList

In [ ]:
encodedList = []
# print(len(decodedList)

for index, X_test in enumerate(testList):
    encoded_stocks = encoder.predict(X_test)
    encodedList.append(encoded_stocks)
    # predicted_stock_price = scaler.inverse_transform(decoded_stocks)

In [ ]:
print(len(encodedList))

359


Κάνουμε iterate την encodedList και παίρνουμε τα encoded time series τα κάνουμε dataframe μιας σειράς και το προσθέτουμε σε μια λίστα απο dataframe. Αυτή τη λίστα την κάνουμε το dataframe result.

Προσθέτουμε το id ως πρώτη στήλη χωρίζουμε το dataframe σε query και dataset και τα κάνουμε export ως .csv αρχεία

In [ ]:
listDf = []

for d in encodedList:

  mylist = []
  for i_1 in range(d.shape[0]):
    for i_2 in range(d.shape[1]):
      mylist.append(d[i_1][i_2])

  temp = pd.DataFrame(mylist).transpose()

  listDf.append(temp)
  # print(d.shape)
  # break

In [ ]:
result = pd.concat(listDf)

In [ ]:
result = result.reset_index(drop=True)

In [ ]:
result.insert(loc = 0, column = 'id', value= df.iloc[:, 0])

In [ ]:
result.columns = [''] * len(result.columns)

In [ ]:
print(result)

                                         ...                                        
0       a -0.107236 -0.078888  0.043876  ... -0.053835 -0.115001 -0.107530  0.064362
1      aa -0.164724 -0.155784 -0.286563  ...  0.021934 -0.150753 -0.141131  0.050911
2    aaba -0.107844 -0.106639 -0.162765  ...  0.052766 -0.119900 -0.084919  0.071203
3    aapl -0.128379 -0.083814 -0.144512  ... -0.089369 -0.101176 -0.107563 -0.011560
4     abc -0.126678 -0.115397 -0.290557  ... -0.199680 -0.099392 -0.157886 -0.213279
..    ...       ...       ...       ...  ...       ...       ...       ...       ...
354  xray -0.141159 -0.107653 -0.069774  ...  0.001934 -0.054113 -0.092601 -0.076725
355   xrx -0.164105 -0.087905 -0.028718  ... -0.295719 -0.077879 -0.108831 -0.099533
356   yum -0.125796 -0.078161  0.004668  ...  0.024518 -0.080142 -0.075700  0.074785
357   zbh -0.061750 -0.093193  0.018614  ... -0.200614 -0.090493 -0.121503 -0.278972
358  zion -0.154126 -0.103265  0.075190  ... -0.102523 -0.131934 

In [ ]:
query = result.tail(10)
query = query.reset_index(drop=True)
print(query)

                                       ...                                        
0     x -0.163578 -0.155128 -0.158787  ... -0.097533 -0.080748 -0.088471 -0.046515
1   xel -0.144926 -0.090476 -0.160701  ... -0.146718 -0.081198 -0.120690 -0.169725
2    xl -0.121942 -0.117945 -0.179730  ... -0.062746 -0.082789 -0.127934 -0.288385
3  xlnx -0.104394 -0.081968  0.034491  ...  0.021050 -0.117004 -0.119502  0.063183
4   xom -0.123246 -0.112879 -0.126322  ... -0.040789 -0.074605 -0.102517 -0.242913
5  xray -0.141159 -0.107653 -0.069774  ...  0.001934 -0.054113 -0.092601 -0.076725
6   xrx -0.164105 -0.087905 -0.028718  ... -0.295719 -0.077879 -0.108831 -0.099533
7   yum -0.125796 -0.078161  0.004668  ...  0.024518 -0.080142 -0.075700  0.074785
8   zbh -0.061750 -0.093193  0.018614  ... -0.200614 -0.090493 -0.121503 -0.278972
9  zion -0.154126 -0.103265  0.075190  ... -0.102523 -0.131934 -0.107607 -0.085785

[10 rows x 2158 columns]


In [ ]:
dataset = result.head(349)
print(dataset)

                                         ...                                        
0       a -0.107236 -0.078888  0.043876  ... -0.053835 -0.115001 -0.107530  0.064362
1      aa -0.164724 -0.155784 -0.286563  ...  0.021934 -0.150753 -0.141131  0.050911
2    aaba -0.107844 -0.106639 -0.162765  ...  0.052766 -0.119900 -0.084919  0.071203
3    aapl -0.128379 -0.083814 -0.144512  ... -0.089369 -0.101176 -0.107563 -0.011560
4     abc -0.126678 -0.115397 -0.290557  ... -0.199680 -0.099392 -0.157886 -0.213279
..    ...       ...       ...       ...  ...       ...       ...       ...       ...
344   whr -0.162897 -0.125499 -0.089821  ... -0.048276 -0.091704 -0.075594 -0.027712
345   wmb -0.157741 -0.133828 -0.144703  ... -0.042606 -0.094301 -0.134888 -0.218213
346   wmt -0.099424 -0.093053 -0.192735  ... -0.276924 -0.105400 -0.132481 -0.138139
347   wor -0.158021 -0.087608  0.030832  ... -0.095690 -0.116479 -0.100193  0.073800
348    wy -0.134535 -0.117630 -0.171392  ... -0.191920 -0.080215 

In [ ]:
dataset.to_csv("/content/gdrive/MyDrive/dataset1.csv", sep="\t", index=False, header=False)

In [ ]:
query.to_csv("/content/gdrive/MyDrive/query1.csv",  sep="\t", index=False, header=False)